# Fine-tuning LLM with TRL using GRPO

_Authored by: [Sergio Paniego](https://github.com/sergiopaniego)_


# 1. Install Dependencies

Let’s start by installing the essential libraries we’ll need for fine-tuning! 🚀


In [2]:
!pip install  -U -q transformers trl datasets peft accelerate
# Tested with transformers==4.48.1, trl==0.14.0.dev0, datasets==3.2.0, peft==0.14.0, accelerate==1.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requi

In [3]:
!pip install git+https://github.com/huggingface/trl.git@main

  Cloning https://github.com/huggingface/trl.git (to revision main) to /tmp/pip-req-build-9ywzwq3n
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-9ywzwq3n
  Resolved https://github.com/huggingface/trl.git to commit fe4b5efe4e23f4331ba9c5b0c8bd92dc8302c287
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.14.0.dev0-py3-none-any.whl size=306806 sha256=39d8f9e5e5f19d6a6006491ca47264a846d84cac5930c8606f332e54fd2711b0
  Stored in directory: /tmp/pip-ephem-wheel-cache-ba9juo4y/wheels/86/55/e9/4fb51fd8f4973abd44ac9118a3cf4610b1271263c00f8f85c9
Successfully built trl
  Attempting uninstall: trl
    Found existing installation: trl 0.13.0
    Uninstalling trl-0.13.0:
      Successfully uninstalled trl-0.13.0


In [10]:
!pip install -q flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 79.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Authenticate with your Hugging Face account to save and share your model directly from this notebook 🗝️.

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import wandb
wandb.login(relogin=True)

# 2. Load Dataset 📁


In [2]:
from datasets import load_dataset

dataset_id = "trl-lib/tldr"
train_dataset, eval_dataset, test_dataset = load_dataset(dataset_id, split=['train[:10%]', 'validation[:10%]', 'test[:10%]'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
train_dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 11672
})

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "Qwen/Qwen2-0.5B-Instruct"

# 4. Fine-Tune the Model using TRL


In [5]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
)

model = get_peft_model(model, lora_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.print_trainable_parameters()

trainable params: 540,672 || all params: 494,573,440 || trainable%: 0.1093


In [6]:
from transformers import AutoModelForSequenceClassification

reward_model_id = "weqweasdas/RM-Gemma-2B"

reward_model = AutoModelForSequenceClassification.from_pretrained(
    reward_model_id,
    num_labels=1,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

'\nreward_model = AutoModelForSequenceClassification.from_pretrained(\n    reward_model_id, \n    num_labels=1, \n    torch_dtype=torch.bfloat16,\n    quantization_config=bnb_config,\n    _attn_implementation="flash_attention_2",\n)\n'

![Image](https://huggingface.co/datasets/trl-lib/documentation-images/resolve/main/grpo_visual.png)

In [7]:
from trl import GRPOConfig

# Configure training arguments using SFTConfig
training_args = GRPOConfig(
    output_dir="Qwen2-0.5B-GRPO",
    learning_rate=1e-5,
    #logging_steps=10,
    #gradient_accumulation_steps=16,
    #max_completion_length=128,
    logging_steps=2,
    gradient_accumulation_steps=2,
    max_completion_length=16,
    num_generations=2,
    #report_to=["tensorboard"]
    report_to=["wandb"]
)

## 4.3 Training the Model 🏃

In [8]:
from trl import GRPOTrainer

trainer = GRPOTrainer(
    model=model,
    reward_model=reward_model,
    args=training_args,
    train_dataset=train_dataset,
)

Time to Train the Model! 🎉

In [9]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
2,0.000000
4,0.000000
6,0.000000
8,0.000100
10,0.000000
12,0.000100
14,0.000100
16,0.000100
18,0.000200
20,0.000100


KeyboardInterrupt: 

Let's save the results 💾

In [ ]:
#trainer.save_model(training_args.output_dir)
trainer.save_model(training_args.output_dir)
trainer.push_to_hub(dataset_name=dataset_id)